## 4. Das neuronale Netzwerk trainieren und testen


Jetzt kommen wir endlich zum Herzstück des Projekts: Das neuronale Netzwerk

Allgemeinere Dokumentation für diese Art von Maschinellem Lernen mit Scikit-learn findet sich [hier](https://scikit-learn.org/stable/modules/neural_networks_supervised.html).


Hier kommt wieder eine neue Library ins Spiel:
[`Scikit-learn`](https://de.wikipedia.org/wiki/Scikit-learn) (ehemals scikits.learn) ist eine Bibliothek zum maschinellen Lernen.


In [ ]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.svm import SVC
import numpy as np


mode = 1
# 1 = Nationalratswahlen
# 2 = Ständeratswahlen

filename_csv_all = "nationalrat_all.csv" if mode == 1 else "ständerat_all.csv"
filename_csv_train = "nationalrat_train.csv" if mode == 1 else "ständerat_train.csv"
filename_csv_test = "nationalrat_test.csv" if mode == 1 else "ständerat_test.csv"

Als erstes müssen die Trainingsdaten in Inputs und Labels aufgeteilt werden. Die Inputs sind in diesem Fall die Antworten zwischen 0 und 1 und die Labels sind die Parteien. Das Netzwerk soll dann lernen, in den Antworten Muster zu finden die auf die Parteien schliessen lassen.

Vielleicht haben Sie schon bemerkt, dass die Parteinamen aktuell noch Text sind. Eigentlich müssten sie deshalb noch in Zahlen umgewandelt werden. [`Scikit-learn`](https://de.wikipedia.org/wiki/Scikit-learn) macht das jedoch automatisch.

---

Für die Interessierten:

Eine Möglichkeit wäre, jeder Partei eine Zahl zuzuweisen, also zB. `Piratenpartei=0, SP=0.1, Grüne=0.2`, jedoch macht das aus mathematischer Sicht keinen Sinn, da die Parteien keine intrinsische Reihenfolge haben. Besser ist es, jeder kandidierenden Person einen Eintrag für jede Partei zu erstellen. Dieser Eintrag ist `0`, wenn die Person nicht in der Partei ist und `1`, wenn sie es ist. Da jede Person in genau einer Partei ist (zumindest nach Datensatz von Smartvote), hat jede Person genau eine Eins unter vielen Nullen, deshalb wird diese Art von Liste - auf mathematisch Vektor - One-Hot-Vector oder zu Deutsch [1-aus-n-Code](https://de.wikipedia.org/wiki/1-aus-n-Code) genannt, wobei n in diesem Fall für die Anzahl Parteien steht.


In [ ]:
def csv_to_vector(filename):
    training_df = pd.read_csv(filename)

    input_columns = [str(q) for q in range(32214, 32289)]
    input_columns.append("gender")

    inputs = training_df[input_columns]
    labels = training_df[["partyAbbreviation"]]

    return inputs, labels


training_inputs, training_labels = csv_to_vector(filename_csv_train)
training_inputs

Jetzt kann endlich das Netzwerk trainiert werden. Wir verwenden ein [Multi-Layer-Perzeptron](https://de.wikipedia.org/wiki/Multi-Layer-Perzeptron) als Modell. Mit den `hidden_layer_sizes` muss experimentiert werden, bis die Genauigkeit zufriedenstellend ist.


In [ ]:
model = MLPClassifier(
    solver="adam",
    hidden_layer_sizes=(20, 30),
    activation="relu",
    verbose=1,
    max_iter=1000,
    tol=0.000001,
)

model.classes_ = np.array(set(pd.read_csv(filename_csv_all)["partyAbbreviation"]))

model.fit(training_inputs, training_labels)

Testen wir doch gleich noch die Genauigkeit. Es ist beim Testen wichtig, dass keine Daten verwendet werden, die das Modell beim Trainieren schon gesehen hat.


In [ ]:
test_inputs, test_labels = csv_to_vector(filename_csv_test)

model.score(test_inputs, test_labels)

~0.6 = 60%

Das heisst, dass etwa 60% der Kandidierenden zur richtigen Partei zugeordnet wurden!

Welche Parteien wurden denn Verwechselt?


In [ ]:
import numpy as np

test_predictions = model.predict(test_inputs)

cm = confusion_matrix(test_labels, test_predictions, labels=model.classes_)
plt.figure(dpi=200)
ax = plt.subplot()
sns.set_theme(font_scale=0.5)
sns.heatmap(cm, annot=True, ax=ax, cmap="Blues", fmt="g", vmax=np.max(cm)/2);


ax.set_xlabel('Vorhergesagte Partei');
ax.set_ylabel('Tatsächliche Partei');
plt.xticks(rotation=45)
plt.yticks(rotation=45)
ax.set_title('Confusion Matrix');
ax.xaxis.set_ticklabels(model.classes_,rotation_mode='anchor', ha="right", va="center");
ax.yaxis.set_ticklabels(model.classes_,rotation_mode='anchor',  ha="right", va="center");
plt.show()

Die Diagonale steht für die richtig vorhergesagten Parteien, alle anderen Verfärbungen sind Verwechslungen. Man kann erkennen, dass viele Verwechslungen zwischen Parteien mit ähnlichen Agenden stattfinden. So werden Jungparteien oftmals mit ihren Mutterparteien verwechselt. Man kann auch andere spannende Dinge beobachten. So gibt es Parteien, die kaum mit anderen verwechselt werden - z.B. die SP - und es gibt solche die sehr oft mit anderen verwechselt werden - z.B. die Mitte und die Piratenpartei. Vielleicht könnten wir versuchen, das zu formalisieren. 

In [ ]:
def print_dict(d: dict):
    maxlen_keys = max(map(len, map(str, d.keys())))
    maxlen_values = max(map(len, map(str, d.values())))
    for key in d.keys():
        print(f"{key:.<{maxlen_keys}}{d[key]:.>{maxlen_values+1}}")


print_dict(
    dict(
        (str(party_name), f"{f_score:.0%}")
        for party_name, f_score in sorted(
            zip(model.classes_, f1_score(test_labels, test_predictions, average=None)),key=lambda x: x[1], reverse=True
        )
    )
)